In [4]:
# Install necessary libraries.
!pip install langchain langchain-google-genai duckduckgo-search requests beautifulsoup4 python-dateutil

# Import necessary libraries.
import time
import os
import json
import requests
import logging
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional
from dateutil import parser as date_parser
from bs4 import BeautifulSoup

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate
from langchain.tools import tool
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import SystemMessage
from duckduckgo_search import DDGS

# Configure logging.
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize enhanced LLM with parameters and rate limiting.
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key="YOUR_KEY_HERE",  # Replace with your key.
    temperature=0.3,  # Lower temperature for more consistent results.
    max_tokens=1024,  # Reduce to help with potential token limits.
    max_retries=2,
    request_timeout=30
)

# Enhanced tools with error handling and functionality.
@tool
def advanced_calculator(expression: str) -> str:

    # Evaluates mathematical expressions safely, supports basic math operations,
    # trigonometric functions, and common mathematical constants.
    # Example: 'sin(3.14159/2)' or '2**3 + sqrt(16)'.

    import math

    # Safe evaluation context with math module included.
    safe_dict = {
        "__builtins__": {},
        "abs": abs, "round": round, "min": min, "max": max,
        "sum": sum, "pow": pow, "divmod": divmod,
        "sin": math.sin, "cos": math.cos, "tan": math.tan,
        "asin": math.asin, "acos": math.acos, "atan": math.atan,
        "sqrt": math.sqrt, "log": math.log, "log10": math.log10,
        "exp": math.exp, "pi": math.pi, "e": math.e,
        "ceil": math.ceil, "floor": math.floor,
        "factorial": math.factorial,
        "math": math  # Include math module for math.pi references.
    }

    try:
        # Clean up the expression.
        expression = expression.strip("'\"")
        expression = expression.replace("^", "**")

        # Handle pi references consistently.
        expression = expression.replace("pi", "math.pi")

        result = eval(expression, safe_dict)

        # Format the result.
        if isinstance(result, float):
            if result.is_integer():
                result = int(result)
            else:
                result = round(result, 6)

        return f"The calculation result is: {result}"
    except Exception as e:
        return f"Error evaluating expression '{expression}': {str(e)}"

@tool
def enhanced_web_search(query: str) -> str:

    # Searches the web and returns detailed results with titles, descriptions, and URLs.
    # Format: 'search_term' or 'search_term|max_results' (e.g., 'python tutorials|10').
    # Default max_results is 3 if not specified (reduced for quota management).

    try:
        # Parse query for optional max_results parameter.
        if '|' in query:
            search_query, max_results_str = query.split('|', 1)
            try:
                max_results = int(max_results_str.strip())
                max_results = min(max_results, 5)  # Cap at 5 for performance.
            except ValueError:
                max_results = 3
                search_query = query  # Use original query if parsing fails.
        else:
            search_query = query
            max_results = 3

        with DDGS() as ddgs:
            results = list(ddgs.text(search_query, max_results=max_results))

        if not results:
            return f"No search results found for: {search_query}"

        formatted_results = []
        for i, result in enumerate(results, 1):
            title = result.get('title', 'No title')
            url = result.get('href', 'No URL')
            snippet = result.get('body', 'No description')

            formatted_results.append(
                f"{i}. **{title}**\n"
                f"   URL: {url}\n"
                f"   Description: {snippet[:150]}{'...' if len(snippet) > 150 else ''}\n"
            )

        return f"Search results for '{search_query}':\n\n" + "\n".join(formatted_results)

    except Exception as e:
        logger.error(f"Web search error: {e}")
        return f"Error performing web search: {str(e)}"

@tool
def web_content_extractor(url: str) -> str:

    # Extracts and returns the main text content from a webpage.
    # Useful for getting detailed information from search results.

    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # Remove script and style elements.
        for script in soup(["script", "style"]):
            script.decompose()

        # Extract main content.
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)

        # Limit content length.
        if len(text) > 1500:
            text = text[:1500] + "... [Content truncated]"

        return f"Content from {url}:\n\n{text}"

    except Exception as e:
        return f"Error extracting content from {url}: {str(e)}"

@tool
def file_manager(command: str) -> str:

    # File operations using command format: 'operation:filename:content'.
    # Operations: read:filename, write:filename:content, append:filename:content, delete:filename, list:directory.
    # Examples: 'read:test.txt', 'write:output.txt:Hello World', 'list:.' or 'list:'.

    try:
        parts = command.split(':', 2)
        if len(parts) < 2:
            return "Invalid format. Use: operation:filename:content (content optional for read/delete/list)"

        operation = parts[0].strip().lower()
        filename = parts[1].strip() if parts[1] else "."
        content = parts[2] if len(parts) > 2 else ""

        if operation == "read":
            if not os.path.exists(filename):
                return f"File '{filename}' does not exist."
            with open(filename, 'r', encoding='utf-8') as file:
                file_content = file.read()
                return f"Content of '{filename}':\n{file_content}"

        elif operation == "write":
            if not content:
                return "Write operation requires content. Use: write:filename:content"
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(content)
            return f"Successfully wrote content to '{filename}'"

        elif operation == "append":
            if not content:
                return "Append operation requires content. Use: append:filename:content"
            with open(filename, 'a', encoding='utf-8') as file:
                file.write(content)
            return f"Successfully appended content to '{filename}'"

        elif operation == "delete":
            if os.path.exists(filename):
                os.remove(filename)
                return f"Successfully deleted '{filename}'"
            else:
                return f"File '{filename}' does not exist."

        elif operation == "list":
            directory = filename if filename and filename != "." else "."
            if os.path.isdir(directory):
                files = os.listdir(directory)
                return f"Files in '{directory}':\n" + "\n".join(files)
            else:
                return f"Directory '{directory}' does not exist."

        else:
            return f"Invalid operation '{operation}'. Use: read, write, append, delete, list"

    except Exception as e:
        return f"File operation error: {str(e)}"

@tool
def date_time_tool(command: str) -> str:

    # Handles date and time operations using command format: 'operation:parameter'.
    # Operations: current, parse:date_string, diff:date1,date2, format:date_string.
    # Examples: 'current', 'parse:2024-01-15', 'diff:2024-01-01,2024-12-31', 'format:2024-01-15'.

    try:
        parts = command.split(':', 1)
        operation = parts[0].strip().lower()
        parameter = parts[1].strip() if len(parts) > 1 else ""

        if operation == "current":
            now = datetime.now()
            return f"Current date and time: {now.strftime('%Y-%m-%d %H:%M:%S')}"

        elif operation == "parse":
            if not parameter:
                return "Please provide a date string to parse. Use: parse:date_string"
            parsed_date = date_parser.parse(parameter)
            return f"Parsed date: {parsed_date.strftime('%Y-%m-%d %H:%M:%S')}"

        elif operation == "diff":
            if not parameter or ',' not in parameter:
                return "Please provide two dates separated by comma. Use: diff:date1,date2"
            dates = parameter.split(",")
            if len(dates) != 2:
                return "Please provide exactly two dates separated by comma."
            date1 = date_parser.parse(dates[0].strip())
            date2 = date_parser.parse(dates[1].strip())
            diff = abs((date2 - date1).days)
            return f"Difference between the dates: {diff} days"

        elif operation == "format":
            if not parameter:
                date_obj = datetime.now()
            else:
                date_obj = date_parser.parse(parameter)

            formats = {
                "iso": date_obj.isoformat(),
                "us": date_obj.strftime("%m/%d/%Y"),
                "eu": date_obj.strftime("%d/%m/%Y"),
                "long": date_obj.strftime("%B %d, %Y"),
                "short": date_obj.strftime("%b %d, %Y")
            }

            result = "Date formats:\n"
            for fmt_name, fmt_date in formats.items():
                result += f"  {fmt_name}: {fmt_date}\n"

            return result

        else:
            return f"Invalid operation '{operation}'. Use: current, parse, diff, format"

    except Exception as e:
        return f"Date/time operation error: {str(e)}"

@tool
def data_processor(command: str) -> str:

    # Processes data using command format: 'operation:data'.
    # Operations: count:text, json_parse:json_string, json_format:json_string, stats:text, csv:csv_data.
    # Examples: 'count:Hello world', 'json_parse:{"key":"value"}', 'stats:Some text here'.

    try:
        parts = command.split(':', 1)
        if len(parts) < 2:
            return "Invalid format. Use: operation:data"

        operation = parts[0].strip().lower()
        data = parts[1]

        if operation == "count":
            words = data.split()
            chars = len(data)
            lines = len(data.splitlines())
            return f"Text statistics:\n  Words: {len(words)}\n  Characters: {chars}\n  Lines: {lines}"

        elif operation == "json_parse":
            try:
                parsed = json.loads(data)
                return f"JSON parsed successfully:\n{json.dumps(parsed, indent=2)}"
            except json.JSONDecodeError as e:
                return f"Invalid JSON: {str(e)}"

        elif operation == "json_format":
            try:
                parsed = json.loads(data)
                formatted = json.dumps(parsed, indent=2, sort_keys=True)
                return f"Formatted JSON:\n{formatted}"
            except json.JSONDecodeError as e:
                return f"Invalid JSON: {str(e)}"

        elif operation == "stats":
            words = data.split()
            unique_words = set(word.lower().strip('.,!?;:"()[]{}') for word in words)
            avg_word_length = sum(len(word) for word in words) / len(words) if words else 0

            return (f"Detailed text statistics:\n"
                   f"  Total words: {len(words)}\n"
                   f"  Unique words: {len(unique_words)}\n"
                   f"  Average word length: {avg_word_length:.2f}\n"
                   f"  Characters: {len(data)}\n"
                   f"  Lines: {len(data.splitlines())}")

        elif operation == "csv":
            lines = data.strip().split('\n')
            if not lines:
                return "No CSV data provided"

            headers = lines[0].split(',')
            rows = [line.split(',') for line in lines[1:]]

            result = f"CSV parsed - {len(headers)} columns, {len(rows)} rows:\n"
            result += f"Headers: {', '.join(headers)}\n"
            if rows:
                result += f"First row: {', '.join(rows[0])}\n"
                if len(rows) > 1:
                    result += f"Last row: {', '.join(rows[-1])}\n"

            return result

        else:
            return f"Invalid operation '{operation}'. Use: count, json_parse, json_format, stats, csv"

    except Exception as e:
        return f"Data processing error: {str(e)}"

@tool
def system_info(query: str = "") -> str:

    # Returns system information including current directory, environment variables, and basic system stats.
    # Usage: system_info("") or system_info("basic") - parameter is optional

    try:
        import platform

        # Basic system info that doesn't require psutil.
        info = {
            "platform": platform.system(),
            "platform_version": platform.version(),
            "architecture": platform.architecture()[0],
            "processor": platform.processor() or "Unknown",
            "python_version": platform.python_version(),
            "current_directory": os.getcwd(),
        }

        # Try to get additional info if psutil is available.
        try:
            import psutil
            info.update({
                "cpu_count": psutil.cpu_count(),
                "memory_gb": round(psutil.virtual_memory().total / (1024**3), 2),
                "disk_usage_gb": round(psutil.disk_usage('/').total / (1024**3), 2)
            })
        except ImportError:
            info["note"] = "Install psutil for additional system stats"

        result = "System Information:\n"
        for key, value in info.items():
            result += f"  {key.replace('_', ' ').title()}: {value}\n"

        return result

    except Exception as e:
        return f"Error getting system info: {str(e)}"

# Collect all tools.
tools = [
    advanced_calculator,
    enhanced_web_search,
    web_content_extractor,
    file_manager,
    date_time_tool,
    data_processor,
    system_info
]

# Memory with sliding window.
memory = ConversationBufferWindowMemory(
    k=5,  # Reduced to save quota
    memory_key="chat_history",
    return_messages=True
)

# System message for better agent behavior.
system_message = SystemMessage(content="""
You are a helpful AI assistant with access to various tools. When using tools:

1. Use tools efficiently and only when necessary
2. Always provide a clear final answer after using tools
3. If a tool gives you a result, use that result directly in your response
4. Don't repeat tool calls unnecessarily
5. Be concise but helpful in your responses

Available capabilities:
- Mathematical calculations with scientific functions
- Web search with detailed results
- File operations (read, write, append, delete, list)
- Date and time operations
- Data processing (JSON, CSV, text analysis)
- System information retrieval
""")

# Prompt template for better ReAct agent behavior.
prompt_template = """You are a helpful AI assistant with access to various tools.

TOOLS:
------
You have access to the following tools:

{tools}

IMPORTANT INSTRUCTIONS:
- Use tools efficiently and provide clear final answers
- When you get a result from a tool, use it directly in your Final Answer
- Don't repeat the same tool call multiple times
- If a calculation tool gives you a result, that IS your answer

To use a tool, use this format:

```
Thought: I need to use a tool to help with this request.
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to give to the Human, you MUST use this format:

```
Thought: I have the information needed to answer the question.
Final Answer: [your complete response here]
```

Previous conversation:
{chat_history}

Question: {input}
Thought: {agent_scratchpad}"""

prompt = PromptTemplate.from_template(prompt_template)

# Create the ReAct agent.
react_agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Initialize enhanced agent with AgentExecutor with better settings.
agent = AgentExecutor(
    agent=react_agent,
    tools=tools,
    memory=memory,
    verbose=True,
    max_iterations=3,
    handle_parsing_errors=True,
    return_only_outputs=True,
    early_stopping_method="generate"  # Stop early if possible.
)

class EnhancedAgent:

    # Wrapper class for the agent with additional functionality and quota management.

    def __init__(self, agent):
        self.agent = agent
        self.conversation_log = []
        self.request_count = 0
        self.max_requests_per_session = 20  # Token quota management.

    def run(self, query: str) -> str:

        # Run a query with enhanced error handling, logging, and quota management.

        try:
            # Check quota.
            if self.request_count >= self.max_requests_per_session:
                return "Session quota exceeded. Please restart to continue."

            logger.info(f"Processing query: {query}")
            start_time = time.time()

            # Increment request counter.
            self.request_count += 1

            # Use invoke instead of run for AgentExecutor.
            response = self.agent.invoke({"input": query})

            # Extract the output from the response.
            if isinstance(response, dict):
                result = response.get('output', str(response))
            else:
                result = str(response)

            execution_time = time.time() - start_time

            # Log conversation.
            self.conversation_log.append({
                "timestamp": datetime.now().isoformat(),
                "query": query,
                "response": result,
                "execution_time": execution_time
            })

            logger.info(f"Query completed in {execution_time:.2f} seconds")
            return result

        except Exception as e:
            logger.error(f"Error processing query '{query}': {e}")
            error_msg = str(e)

            # Handle token quota exceeded errors gracefully.
            if "quota" in error_msg.lower() or "429" in error_msg:
                return "API quota exceeded. Please wait before making more requests or restart the session."

            return f"I encountered an error while processing your request: {error_msg}"

    def run_with_retry(self, query: str, max_retries: int = 2) -> str:

        # Run a query with retry logic for handling transient errors.

        for attempt in range(max_retries + 1):
            try:
                result = self.run(query)
                if "quota exceeded" not in result.lower():
                    return result
                else:
                    if attempt < max_retries:
                        logger.info(f"Quota exceeded, waiting before retry {attempt + 1}/{max_retries}")
                        time.sleep(5)  # Wait before retry.
                    else:
                        return result
            except Exception as e:
                if attempt < max_retries:
                    logger.info(f"Attempt {attempt + 1} failed, retrying...")
                    time.sleep(2)
                else:
                    return f"Failed after {max_retries + 1} attempts: {str(e)}"

        return "Unexpected error in retry logic"

    def get_conversation_history(self) -> List[Dict]:

        # Return the conversation history.

        return self.conversation_log

    def save_conversation(self, filename: str = "conversation_log.json"):

        # Save conversation history to file.

        try:
            with open(filename, 'w') as f:
                json.dump(self.conversation_log, f, indent=2)
            return f"Conversation saved to {filename}"
        except Exception as e:
            return f"Error saving conversation: {str(e)}"

    def reset_quota(self):

        # Reset the request counter.

        self.request_count = 0
        logger.info("Request quota reset")

# Test the enhanced agent with better quota management.
if __name__ == "__main__":
    # Create enhanced agent instance.
    enhanced_agent = EnhancedAgent(agent)

    # Create example files for testing.
    if not os.path.exists("example.txt"):
        with open("example.txt", "w") as f:
            f.write("Hello, this is a test file!\nIt contains some text for the file_manager tool to read.\nThis is line 3.")

    # Comprehensive test queries - prioritized for testing the most important fixes first.
    test_queries = [
        "What is the square root of 144 plus sin(pi/2)?",
        "Calculate the difference in days between 2024-01-01 and 2024-12-31 using date_time_tool diff:2024-01-01,2024-12-31",
        "Count words in this text using data_processor count:The quick brown fox jumps over the lazy dog",
        "Process this JSON data using data_processor json_parse:{\"name\": \"John\", \"age\": 30, \"city\": \"New York\"}",
        "Read the content of example.txt using file_manager with read:example.txt",
        "Get the current date and time using date_time_tool with current",
        "Show me system information",
        "Search for recent developments in artificial intelligence",
        "Create a file called 'test_output.txt' with content using file_manager write:test_output.txt:Hello from the enhanced agent!",
        "List all files in the current directory using file_manager list:."
    ]

    print("=== Enhanced AI Agent Test Suite (Fixed Version) ===\n")

    for i, query in enumerate(test_queries, 1):
        print(f"Test {i}/{len(test_queries)}: {query}")
        print("-" * 80)

        try:
            response = enhanced_agent.run_with_retry(query)
            print(f"Response: {response}\n")
        except Exception as e:
            print(f"Error: {e}\n")

        # Increased delay to avoid API limits.
        time.sleep(5)

        # Check if model should stop due to token quota issues.
        if "quota exceeded" in str(response).lower():
            print("Stopping tests due to quota limits. Consider running fewer tests or waiting longer between requests.")
            break

    # Save conversation log.
    try:
        enhanced_agent.save_conversation()
        print("Conversation log saved to conversation_log.json")
    except Exception as e:
        print(f"Error saving conversation log: {e}")

    print(f"\nTotal requests made: {enhanced_agent.request_count}")
    print("Test suite completed!")

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: WARNING! request_timeout is not default parameter.
                request_timeout was transferred to model_kwargs.
                Please confirm that request_timeout is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


=== Enhanced AI Agent Test Suite (Fixed Version) ===

Test 1/10: What is the square root of 144 plus sin(pi/2)?
--------------------------------------------------------------------------------


> Entering new AgentExecutor chain...
Thought: I need to use a tool to calculate the square root of 144 plus sin(pi/2).
Action: advanced_calculator
Action Input: 'sqrt(144) + sin(3.14159/2)'The calculation result is: 13.0Thought: I have the information needed to answer the question.
Final Answer: 13.0


> Finished chain.
Response: 13.0

Test 2/10: Calculate the difference in days between 2024-01-01 and 2024-12-31 using date_time_tool diff:2024-01-01,2024-12-31
--------------------------------------------------------------------------------


> Entering new AgentExecutor chain...
Thought: I need to use the date_time_tool to calculate the difference in days between the two dates.
Action: date_time_tool
Action Input: diff:2024-01-01,2024-12-31Difference between the dates: 365 daysThought: I have t

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 8
}
].


Thought: I need to use the file_manager tool to create and write to the file 'test_output.txt'.
Action: file_manager
Action Input: write:test_output.txt:Hello from the enhanced agent!Successfully wrote content to 'test_output.txt'

ERROR:__main__:Error processing query 'Create a file called 'test_output.txt' with content using file_manager write:test_output.txt:Hello from the enhanced agent!': 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 6
}
]




> Entering new AgentExecutor chain...
Thought: I need to use the file_manager tool to create and write to the file 'test_output.txt'.
Action: file_manager
Action Input: write:test_output.txt:Hello from the enhanced agent!Successfully wrote content to 'test_output.txt'Thought: I have the information needed to answer the question.
Final Answer: The file 'test_output.txt' has been created with the content "Hello from the enhanced agent!".


> Finished chain.
Response: The file 'test_output.txt' has been created with the content "Hello from the enhanced agent!".

Test 10/10: List all files in the current directory using file_manager list:.
--------------------------------------------------------------------------------


> Entering new AgentExecutor chain...
Thought: I need to use the file_manager tool to list all files in the current directory.
Action: file_manager
Action Input: list:.Files in '.':
.config
example.txt
test_output.txt
conversation_log.json
sample_dataThought: I have the 